In [ ]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt

In [ ]:
accident_table = pd.read_csv('data.csv')

In [ ]:
accident_table.info()

In [ ]:
accident_table['State'].describe()